### When a function takes a list of objeccts as a parameter, it's often important to iterate over that list multiple times.

In [3]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits=[15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [7]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

it = read_visits('my_numbers.txt')
#print(list(it))
percentages = normalize(it)
print(percentages)

[]


This behavior occurs because an iterator produces its results only a single time. If you iterate over an iterator or a generator that hs already raised a *StopIteration exception*, you won't get any results the second time around

In [8]:
it =read_visits('my_numbers.txt')
print(list(it))
print(list(it))  # Already exhausted

[15, 35, 80]
[]


In [10]:
#To solve this problem, you can explicitly exhaust an input iterator
# and keep a copy of its entire contents in a list.
def normalize_copy(numbers):
    numbers_copy = list(numbers)  # Copy the iterator
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [11]:
#One way around this is to accept a function that returns a new iterator each time it's called
def normalize_func(get_iter):
    total = sum(get_iter())   # New iterator
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result
# To use normalize_func, I can pass in a lambda expression that calls the generator and produces a new iterator each time
path = 'my_numbers.txt'
percentages = normalize_func(lambda: read_visits(path))
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


Although this works, having to pass a lambda function like this is clumsy. A better way to achieve the same result is to provide a new container class that implements the iterator protocol.

In [13]:
#you can achieve all of this behavior for your classes by implementing the __iter__
#method as a generator.

class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [19]:
#you can test an input value for htis behavior and raise a TypeError to reject arguments that can't be repeatedly iterated over
def normalize_defensive(numbers):
    if iter(numbers) is numbers: # An iterator -- bad!
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value /total
        result.append(percent)
    return result

In [25]:
from collections.abc import Iterator

def normalize_defensive(numbers):
    if isinstance(numbers, Iterator):  #Another way to check
        raise TypeError('must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize_defensive(visits)
print(percentages)
assert sum(percentages) == 100.0

visits = ReadVisits(path)
percentages = normalize_defensive(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [26]:
#The function raises an exception if the input is an iterator rather than a container
visits = [15, 35, 80]
it = iter(visits)
percentages = normalize_defensive(it)
print(percentages)
assert sum(percentages) == 100.0

TypeError: must supply a container